## Prevent random disconnects

This cell runs JS code to automatic reconnect to runtime.

In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

## Check GPU

*   Google Colab can provide you with P100 Tesla graphics card everytime
*   Here you can check the model of GPU before using EverybodyDanceNow


In [ ]:
!nvidia-smi

## Mount and Import from Drive

*   You can import dataset or individual data, like model files with Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd "/content/drive/My Drive/Colab_Files/EverybodyDanceNow/"

## List the Checkpoint files from Drive

*   List the trained Checkpoint files from Drive

In [ ]:
%cd ./model_local
%ls

## Install some dependencies
*   Install Python Dependencies 

In [ ]:
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install --user --upgrade tensorflow  
!pip install dominate

## Train Global Stage Model
*   Feel free to edit the path and other parameters accordingly by clicking this cell

In [ ]:
!python3 train_fullts.py \
--name ./model_global \
--dataroot ./dataset/train/ \
--checkpoints_dir ./checkpoints/ \
--loadSize 512 \
--no_instance \
--no_flip \
--tf_log \
--label_nc 6 \
--save_latest_freq 100 \

## Train Local Stage Model
*   Feel free to edit the path and other parameters accordingly by clicking this cell

In [ ]:
!python3 train_fullts.py \
--name ./model_local/ \
--dataroot ./dataset/train/  \
--checkpoints_dir ./checkpoints/ \
--loadSize 1024 \
--no_instance \
--no_flip \
--tf_log \
--netG local \
--ngf 32 \
--num_D 3 \
--label_nc 6 \
--resize_or_crop none \
--save_latest_freq 100 \
--save_epoch_freq 1 \
--print_freq 10 \
--continue_train  
#--load_pretrain ./checkpoints/model_local/ \


## Train Local Stage Model with Face Generator and Discriminator
*   Feel free to edit the path and other parameters accordingly by clicking this cell

In [ ]:
!python3 train_fullts.py \
--name ./model_local/ \
--dataroot ./dataset/train/  \
--checkpoints_dir ./checkpoints/ \
--load_pretrain ./checkpoints/model_local/ \
--loadSize 1024 \
--face_discrim \
--face_generator \
--faceGtype global \
--niter_fix_main 10 \
--netG local \
--ngf 32 \
--num_D 3 \
--resize_or_crop none \
--no_instance \
--no_flip \
--tf_log \
--label_nc 6 \
--save_latest_freq 100 \
--continue_train \

## Test Local Stage Model with Face Generator and Discriminator
*   Feel free to edit the path and other parameters accordingly by clicking this cell

In [ ]:
%cd "/content/drive/My Drive/Colab_Files/EverybodyDanceNow/"

!python3 test_fullts.py \
--name model_local \
--dataroot ./dataset/test/ \
--load_pretrain ./checkpoints/model_local \
--results_dir ./result/ \
--loadSize 1024 \
--face_generator \
--faceGtype global \
--netG local \
--ngf 32 \
--resize_or_crop none \
--no_instance \
--how_many 6000 \
--label_nc 6

## Test Local Stage Model without Face Generator and Discriminator
*   Feel free to edit the path and other parameters accordingly by clicking this cell

In [ ]:
%cd "/content/drive/My Drive/Colab_Files/EverybodyDanceNow/"

!python3 test_fullts.py \
--name model_local \
--dataroot ./dataset/test/ \
--checkpoints_dir ./checkpoints/ \
--results_dir ./result/ \
--loadSize 1024 \
--netG local \
--ngf 32 \
--resize_or_crop none \
--no_instance \
--how_many 10000 \
--label_nc 6

## Generate images and convert them into result video
*   Umm...duh

In [ ]:
%cd "/content/drive/My Drive/Colab_Files/EverybodyDanceNow/"

import glob
import os
import re
import sys

def get_trailing_number(s):
    m = re.search(r'\d+$', s)
    return int(m.group()) if m else None
  
# Function to rename multiple files 
def main(mypath, name_stub, len_num, remove): 
	count = 0 
	files = glob.glob(mypath+"*.json")
	print(files)
	for current in files:
		number_trailing = re.findall('\d+',current )
		number_trailing = [s.lstrip("0") for s in number_trailing]
		
		#print(number_trailing)
		if remove==True:
			os.rename(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+"_keypoints")#+".json") 
			print(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+"_keypoints")
		else:
			os.rename(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+"_keypoints"+".json") 
			print(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+"_keypoints"+'.json')
		count = count + 1

def main_2(mypath, name_stub, len_num, remove): 
	count = 0 
	files = glob.glob(mypath+"*.png")
	print(files)
	for current in files:
		number_trailing = re.findall('\d+',current )
		number_trailing = [s.lstrip("0") for s in number_trailing]
		
		if remove==True:
			os.rename(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num))#+".jpg") 
			print(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num))
		else:
			os.rename(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+".png") 
			print(current, mypath+name_stub+str(number_trailing[0]).zfill(len_num)+".png") 
		count = count + 1
  
# Driver Code 
# Calling main() function, run command ffmpeg -i output_%4d.png  output.avi
main_2("./result/model_local/test_latest/images/", "output_",4, False) 
#main_2("./dataset/test/test_label/", "frame",4, False) 
#!apt install ffmpeg
#!ffmpeg -i ./dataset/test/test_label/frame%04d.png -framerate 24n ./result/label_test.mp4
!ffmpeg -framerate 24 -i ./result/model_local/test_latest/images/output_%04d.png  -b:v 10M ./result/output.mp4
print('done!')

## Flush and Unmount Google Drive
*   Hopefully it's done..open Drive to see the results

In [ ]:
drive.flush_and_unmount()